In [1]:
import h5py
import math
import numpy as np
from operator import itemgetter
import scipy.stats as st
from itertools import compress

In [2]:
filename = 'all_matrices.hdf5'
all_matrices = h5py.File(filename, 'r')

In [3]:
fullmat0123 = all_matrices[all_matrices.keys()[0]]['0123']
fullmat0213 = all_matrices[all_matrices.keys()[0]]['0213']
fullmat0312 = all_matrices[all_matrices.keys()[0]]['0312']

In [4]:
# get the scores
scores = [math.sqrt(np.sum(np.square(np.linalg.svd(fullmat0123)[1][10:15]))),math.sqrt(np.sum(np.square(np.linalg.svd(fullmat0213)[1][10:15]))),math.sqrt(np.sum(np.square(np.linalg.svd(fullmat0312)[1][10:15])))]
scores
# get index of minimum score, via <https://stackoverflow.com/questions/2474015/getting-the-index-of-the-returned-max-or-min-item-using-max-min-on-a-list>
min_index, min_value = min(enumerate(scores), key=itemgetter(1))
# get winning matrix
winning_mat = all_matrices[all_matrices.keys()[0]][['0123','0213','0312'][min_index]]

In [22]:
#0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15
#00,01,02,03,10,11,12,13,20,21,22,23,30,31,32,33

def calcHils(invmat):
    comb_dict = dict(zip([00,01,02,03,10,11,12,13,20,21,22,23,30,31,32,33], [0,  1,  2,  3,  4,  5,  6,  7,  8,  9,  10, 11, 12, 13, 14, 15]))
    num_iijj = (invmat[comb_dict[00],comb_dict[11]] + invmat[comb_dict[00],comb_dict[22]] +
        invmat[comb_dict[00],comb_dict[33]] + invmat[comb_dict[11],comb_dict[00]] + invmat[comb_dict[11],comb_dict[22]] +
        invmat[comb_dict[11],comb_dict[33]] + invmat[comb_dict[22],comb_dict[00]] + invmat[comb_dict[22],comb_dict[11]] +
        invmat[comb_dict[22],comb_dict[33]] + invmat[comb_dict[33],comb_dict[00]] + invmat[comb_dict[33],comb_dict[11]] +
        invmat[comb_dict[33],comb_dict[22]])

    num_ijji = (invmat[comb_dict[01],comb_dict[10]] + invmat[comb_dict[02],comb_dict[20]] +
        invmat[comb_dict[03],comb_dict[30]] + invmat[comb_dict[10],comb_dict[01]] + invmat[comb_dict[12],comb_dict[21]] +
        invmat[comb_dict[13],comb_dict[31]] + invmat[comb_dict[20],comb_dict[02]] + invmat[comb_dict[21],comb_dict[12]] +
        invmat[comb_dict[23],comb_dict[32]] + invmat[comb_dict[30],comb_dict[03]] + invmat[comb_dict[31],comb_dict[13]] +
        invmat[comb_dict[32],comb_dict[23]])

    num_ijij = (invmat[comb_dict[01],comb_dict[01]] + invmat[comb_dict[02],comb_dict[02]] +
        invmat[comb_dict[03],comb_dict[03]] + invmat[comb_dict[10],comb_dict[10]] + invmat[comb_dict[12],comb_dict[12]] +
        invmat[comb_dict[13],comb_dict[13]] + invmat[comb_dict[20],comb_dict[20]] + invmat[comb_dict[21],comb_dict[21]] +
        invmat[comb_dict[23],comb_dict[23]] + invmat[comb_dict[30],comb_dict[30]] + invmat[comb_dict[31],comb_dict[31]] +
        invmat[comb_dict[32],comb_dict[32]])
    [num_iijj,num_ijji,num_ijij]
    N = sum(map(sum, invmat))

    # calculate probability, add .05 to counts in case some of them are 0
    p_iijj = (num_iijj + .05)/N
    p_ijji = (num_ijji + .05)/N
    p_ijij = (num_ijij + .05)/N

    f1 = p_iijj - p_ijij
    f2 = p_ijji - p_ijij
    if not(f2):
        f2 = f2 + .000001
    rat_f1_f2 = f1/f2

    var_f1 = (1/N) * ( p_iijj*(1-p_iijj) + p_ijij*(1-p_ijij) + 2*p_iijj*p_ijij )
    var_f2 = (1/N) * ( p_ijji*(1-p_ijji) + p_ijij*(1-p_ijij) + 2*p_ijji*p_ijij )

    cov_f1_f2 = (1/N) * ( -p_iijj*p_ijji + p_iijj*p_ijij + p_ijji*p_ijij + p_ijij*(1-p_ijij))

    H = (f2 * rat_f1_f2) / math.sqrt( var_f2*(rat_f1_f2**2) - 2*cov_f1_f2*rat_f1_f2 + var_f1 )
    return H;
def calcp(z):
    p = st.norm.sf(abs(z))*2
    return p;

In [160]:
# perform only on winning quartet
hils = []
pvals = []
for i in range(len(all_matrices)):
    fullmat0123 = all_matrices[all_matrices.keys()[i]]['0123']
    fullmat0213 = all_matrices[all_matrices.keys()[i]]['0213']
    fullmat0312 = all_matrices[all_matrices.keys()[i]]['0312']
    # get the scores
    scores = [math.sqrt(np.sum(np.square(np.linalg.svd(fullmat0123)[1][10:15]))),math.sqrt(np.sum(np.square(np.linalg.svd(fullmat0213)[1][10:15]))),math.sqrt(np.sum(np.square(np.linalg.svd(fullmat0312)[1][10:15])))]
    scores
    # get index of minimum score, via <https://stackoverflow.com/questions/2474015/getting-the-index-of-the-returned-max-or-min-item-using-max-min-on-a-list>
    min_index, min_value = min(enumerate(scores), key=itemgetter(1))
    # get winning matrix
    winning_mat = all_matrices[all_matrices.keys()[i]][['0123','0213','0312'][min_index]]
    hils_temp = calcHils(winning_mat)
    hils.append(hils_temp)
    pvals.append(calcp(hils_temp))



In [27]:
#perform on all quartets
hils = []
pvals = []
for i in range(len(all_matrices)):
    for p in [0,1,2]:
        curmat = all_matrices[all_matrices.keys()[i]][['0123','0213','0312'][p]]
        hils_temp = calcHils(curmat)
        hils.append(hils_temp)
        pvals.append(calcp(hils_temp))

In [31]:
pvals

[0.16783608493579405,
 0.0,
 0.0,
 0.18055500853496731,
 0.18460950415810651,
 0.18460950415810662,
 0.068115058214988089,
 0.071740005217301131,
 0.071740005217301131,
 0.34050967234622387,
 0.0,
 0.0,
 0.0,
 0.23660469466304768,
 0.23660469466304768,
 2.0959904930875441e-90,
 0.61715009333132165,
 0.61715009333132165,
 4.1655963746016732e-76,
 0.86549982310548623,
 0.86549982310548623,
 9.3479348270375995e-209,
 0.32214296922575192,
 0.32214296922575192,
 0.0,
 0.13819611995893938,
 0.13819611995893938,
 2.8408508558477478e-236,
 0.69483869137999088,
 0.69483869137999088,
 0.0,
 0.39997152982930506,
 0.39997152982930506,
 0.0,
 0.13876856025062639,
 0.13876856025062639,
 0.0,
 0.28888783135765361,
 0.28888783135765339,
 0.0,
 0.60164476314283299,
 0.60164476314283299,
 0.0,
 0.87586159888655357,
 0.87586159888655357,
 0.042938438023154113,
 0.086968511192473646,
 0.086968511192473646,
 0.044277290432559464,
 0.087784451012377612,
 0.087784451012377612,
 5.7562880613792534e-243,
 0.61

In [30]:
    fname = "analysis-ipyrad/min4_outfiles/min4.snps.phy"
    with open(fname) as f:
        snps = f.readlines()
    # remove whitespace characters like `\n` at the end of each line
    snps = [x.strip() for x in snps] 
    snps.pop(0)
    
    #read in map
    fname = "analysis-ipyrad/min4_outfiles/min4.snps.map"
    with open(fname) as f:
        snpmap = f.readlines()
    # remove whitespace characters like `\n` at the end of each line
    snpmap = [x.strip() for x in snpmap] 
    snpmap = [i.split('\t') for i in snpmap]
    snpmap = np.array(snpmap)
    # get rid of inner column, convert to int
    reducedmap = snpmap[:,[0,2,3]].astype(int)
    
    # save names by themselves and make list of corresponding integers
    names = [snps[i][0:27].replace(" ", "") for i in range(len(snps))]

In [229]:
fil = [set(str(all_matrices.keys()[i]).split('_')[1:4]) == set(['1','4','8']) for i in range(len(all_matrices))]
list(compress(all_matrices.keys(),fil))

[u'0_1_4_8']

{'1', '4', '8'}